# 1. Read data

In [ ]:
import pandas as pd
import numpy as np
from collections import defaultdict

In [ ]:
from google.colab import files
data = files.upload()

Saving true_1000_fulldata.csv to true_1000_fulldata.csv


In [ ]:
raw_data = pd.read_csv('true_1000_fulldata.csv')
sa3_num = len(raw_data['SA3 Code'].unique())

In [ ]:
raw_data.head()

,Year,SA3 Code,SA3 Name,m0-4,m5-9,m10-14,m15-19,m20-24,m25-29,m30-34,...,f45-49,f50-54,f55-59,f60-64,f65-69,f70-74,f75-79,f80-84,f85+,Total
0,1991,10101,Goulburn - Yass,2603,2565,2517,2472,2178,2392,2478,...,1859,1591,1412,1382,1252,967,799,493,353,61667
1,1991,10102,Queanbeyan,1593,1362,1223,1406,1743,1803,1670,...,991,745,611,575,544,393,275,177,105,35281
2,1991,10103,Snowy Mountains,741,641,577,591,848,1037,1047,...,504,413,328,320,258,237,152,97,59,18092
3,1991,10104,South Coast,1985,1987,1697,1358,963,1258,1838,...,1318,1370,1651,2166,2075,1437,958,455,314,53440
4,1991,10201,Gosford,5373,5182,4631,4802,3928,4659,5303,...,3709,2884,2884,3567,4002,3573,2661,1565,965,135708


In [ ]:
raw_data.columns

Index(['Year', 'SA3 Code', 'SA3 Name', 'm0-4', 'm5-9', 'm10-14', 'm15-19',
       'm20-24', 'm25-29', 'm30-34', 'm35-39', 'm40-44', 'm45-49', 'm50-54',
       'm55-59', 'm60-64', 'm65-69', 'm70-74', 'm75-79', 'm80-84', 'm85+',
       'f0-4', 'f5-9', 'f10-14', 'f15-19', 'f20-24', 'f25-29', 'f30-34',
       'f35-39', 'f40-44', 'f45-49', 'f50-54', 'f55-59', 'f60-64', 'f65-69',
       'f70-74', 'f75-79', 'f80-84', 'f85+', 'Total'],
      dtype='object')

In [ ]:
raw_data['m_total'] = raw_data.apply(lambda x: x['m0-4']+x['m5-9']+x['m10-14']+x['m15-19']+
                                               x['m20-24']+x['m25-29']+x['m30-34']+x['m35-39']+
                                               x['m40-44']+x['m45-49']+x['m50-54']+x['m55-59']+
                                               x['m60-64']+x['m65-69']+x['m70-74']+x['m75-79']+
                                               x['m80-84']+x['m85+'], axis=1)

In [ ]:
raw_data['f_total'] = raw_data.apply(lambda x: x['f0-4']+x['f5-9']+x['f10-14']+x['f15-19']+
                                               x['f20-24']+x['f25-29']+x['f30-34']+x['f35-39']+
                                               x['f40-44']+x['f45-49']+x['f50-54']+x['f55-59']+
                                               x['f60-64']+x['f65-69']+x['f70-74']+x['f75-79']+
                                               x['f80-84']+x['f85+'], axis=1)

In [ ]:
assert((raw_data['m_total']+raw_data['f_total'] == raw_data['Total']).all())

In [ ]:
raw_data

,Year,SA3 Code,SA3 Name,m0-4,m5-9,m10-14,m15-19,m20-24,m25-29,m30-34,...,f55-59,f60-64,f65-69,f70-74,f75-79,f80-84,f85+,Total,m_total,f_total
0,1991,10101,Goulburn - Yass,2603,2565,2517,2472,2178,2392,2478,...,1412,1382,1252,967,799,493,353,61667,31422,30245
1,1991,10102,Queanbeyan,1593,1362,1223,1406,1743,1803,1670,...,611,575,544,393,275,177,105,35281,17948,17333
2,1991,10103,Snowy Mountains,741,641,577,591,848,1037,1047,...,328,320,258,237,152,97,59,18092,9665,8427
3,1991,10104,South Coast,1985,1987,1697,1358,963,1258,1838,...,1651,2166,2075,1437,958,455,314,53440,27048,26392
4,1991,10201,Gosford,5373,5182,4631,4802,3928,4659,5303,...,2884,3567,4002,3573,2661,1565,965,135708,66257,69451
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6820,2011,80105,North Canberra,1151,1004,901,1753,4064,3371,2345,...,1150,950,752,618,584,490,538,49910,25390,24520
6821,2011,80106,South Canberra,601,607,659,739,897,1281,894,...,859,746,483,393,326,396,498,25176,12382,12794
6822,2011,80107,Tuggeranong,3122,2922,3165,3730,3672,3338,2918,...,3088,2291,1293,889,652,456,364,89319,44209,45110
6823,2011,80108,Weston Creek,873,803,672,679,656,692,724,...,701,935,701,494,344,242,341,23354,11478,11876


In [ ]:
sa3_codes = raw_data['SA3 Code'].unique()

# 2. Split Train, Validation, Test set

In [ ]:
train_data = raw_data[(raw_data['Year']>=1991) & (raw_data['Year']<=2001)]
#val_data = raw_data[(raw_data['Year']>=2002) & (raw_data['Year']<=2005)]
# remove vali 10-10
test_data = raw_data[(raw_data['Year']>=2002) & (raw_data['Year']<=2011)]

In [ ]:
# Split the data by sex
age_groups = ['0-4','5-9', '10-14', '15-19', '20-24', '25-29', '30-34', '35-39','40-44', '45-49', '50-54', '55-59', '60-64', '65-69', '70-74','75-79', '80-84', '85+']
population_m_dict = defaultdict(dict)
population_f_dict = defaultdict(dict)

In [ ]:
# save population for each year into lists
for sa3_code in sa3_codes:
  population_m_dict[sa3_code] = dict()
  population_f_dict[sa3_code] = dict()
  for year in range(1991,2012):
    if(raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)]['Total'].size>0):
      population_m_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][['m0-4',
                              'm5-9', 'm10-14', 'm15-19', 'm20-24', 'm25-29', 'm30-34', 'm35-39',
                              'm40-44', 'm45-49', 'm50-54', 'm55-59', 'm60-64', 'm65-69', 'm70-74',
                              'm75-79', 'm80-84', 'm85+']].values.tolist()[0]
      population_f_dict[sa3_code][year] = raw_data[(raw_data['Year']==year) & (raw_data['SA3 Code']==sa3_code)][['f0-4', 'f5-9', 'f10-14', 'f15-19',
                              'f20-24', 'f25-29', 'f30-34', 'f35-39', 'f40-44', 'f45-49', 'f50-54',
                              'f55-59', 'f60-64', 'f65-69', 'f70-74', 'f75-79', 'f80-84', 'f85+']].values.tolist()[0]

In [ ]:
pd.DataFrame(population_m_dict)

,10101,10102,10103,10104,10201,10202,10301,10302,10303,10304,...,70203,70204,70205,80101,80103,80105,80106,80107,80108,80109
1991,"[2603, 2565, 2517, 2472, 2178, 2392, 2478, 231...","[1593, 1362, 1223, 1406, 1743, 1803, 1670, 139...","[741, 641, 577, 591, 848, 1037, 1047, 890, 743...","[1985, 1987, 1697, 1358, 963, 1258, 1838, 2141...","[5373, 5182, 4631, 4802, 3928, 4659, 5303, 516...","[4332, 4072, 3481, 3469, 2956, 3482, 3957, 364...","[1530, 1588, 1588, 1925, 1635, 1386, 1465, 139...","[2621, 2671, 2586, 2236, 1679, 2062, 2280, 212...","[1866, 1950, 1716, 1437, 1330, 1563, 1680, 153...","[2076, 2133, 2130, 2014, 1805, 1702, 1836, 178...",...,"[814, 670, 577, 580, 704, 677, 614, 526, 387, ...","[687, 766, 618, 554, 476, 516, 589, 549, 465, ...","[937, 873, 645, 622, 868, 1001, 854, 718, 628,...","[3436, 3697, 4225, 4864, 4152, 3485, 3432, 360...","[72, 49, 29, 86, 188, 153, 111, 78, 78, 52, 39...","[1121, 960, 858, 2112, 2759, 1956, 1831, 1436,...","[615, 673, 664, 756, 824, 840, 785, 728, 838, ...","[4388, 4149, 3506, 2915, 2413, 3533, 4038, 388...","[799, 942, 1232, 1638, 1238, 951, 854, 890, 13...","[1021, 1002, 1059, 1445, 1915, 1388, 1268, 122..."
1992,"[2605, 2567, 2499, 2371, 2183, 2294, 2483, 233...","[1625, 1411, 1264, 1375, 1774, 1774, 1741, 147...","[740, 645, 597, 566, 844, 988, 1032, 914, 762,...","[1962, 2038, 1792, 1355, 991, 1204, 1798, 2147...","[5504, 5338, 4807, 4713, 4145, 4551, 5417, 529...","[4503, 4212, 3652, 3443, 3121, 3475, 4111, 380...","[1522, 1582, 1614, 1867, 1694, 1372, 1481, 142...","[2615, 2662, 2575, 2171, 1718, 1999, 2279, 215...","[1885, 1925, 1726, 1408, 1319, 1497, 1680, 154...","[2081, 2132, 2134, 1964, 1857, 1666, 1862, 181...",...,"[810, 697, 583, 595, 714, 672, 606, 521, 393, ...","[692, 751, 624, 546, 499, 511, 583, 540, 455, ...","[970, 905, 669, 652, 872, 1016, 896, 746, 635,...","[3342, 3542, 4019, 4580, 4452, 3395, 3365, 349...","[67, 48, 30, 80, 187, 151, 112, 79, 78, 57, 40...","[1101, 943, 858, 1959, 2914, 1924, 1829, 1436,...","[601, 657, 670, 730, 853, 833, 789, 742, 807, ...","[4646, 4426, 3801, 3061, 2758, 3635, 4347, 419...","[781, 897, 1170, 1479, 1285, 921, 856, 865, 12...","[1012, 1009, 1070, 1337, 1893, 1365, 1296, 123..."
1993,"[2585, 2557, 2480, 2294, 2145, 2183, 2455, 234...","[1638, 1445, 1305, 1348, 1769, 1730, 1785, 155...","[736, 643, 610, 553, 833, 938, 999, 936, 777, ...","[1920, 2069, 1885, 1356, 1002, 1145, 1728, 214...","[5577, 5449, 4979, 4637, 4296, 4420, 5447, 540...","[4620, 4311, 3818, 3418, 3226, 3436, 4189, 393...","[1496, 1570, 1644, 1824, 1722, 1354, 1476, 144...","[2580, 2628, 2562, 2113, 1735, 1931, 2238, 218...","[1889, 1891, 1751, 1384, 1279, 1424, 1663, 155...","[2070, 2116, 2146, 1923, 1870, 1628, 1859, 182...",...,"[806, 724, 590, 600, 721, 671, 610, 512, 412, ...","[695, 737, 636, 534, 521, 510, 577, 529, 455, ...","[993, 926, 702, 672, 874, 1040, 943, 769, 658,...","[3220, 3371, 3849, 4401, 4619, 3303, 3251, 335...","[62, 46, 32, 79, 188, 147, 109, 80, 80, 65, 44...","[1068, 926, 854, 1857, 2975, 1871, 1796, 1423,...","[589, 643, 689, 724, 868, 833, 789, 762, 800, ...","[4728, 4536, 4007, 3190, 2998, 3666, 4490, 431...","[757, 847, 1107, 1361, 1281, 888, 838, 832, 11...","[979, 993, 1071, 1255, 1821, 1342, 1298, 1224,..."
1994,"[2557, 2551, 2466, 2244, 2098, 2087, 2425, 234...","[1643, 1482, 1340, 1334, 1746, 1707, 1834, 161...","[728, 650, 626, 539, 812, 900, 980, 953, 804, ...","[1873, 2113, 1978, 1376, 1010, 1090, 1669, 213...","[5620, 5557, 5133, 4606, 4389, 4310, 5468, 547...","[4716, 4418, 3976, 3429, 3300, 3423, 4275, 404...","[1474, 1562, 1680, 1810, 1737, 1347, 1476, 147...","[2540, 2608, 2550, 2079, 1733, 1885, 2203, 219...","[1881, 1867, 1761, 1374, 1227, 1371, 1639, 156...","[2039, 2093, 2143, 1901, 1855, 1595, 1855, 183...",...,"[798, 739, 602, 615, 724, 664, 606, 513, 427, ...","[710, 725, 647, 533, 542, 506, 567, 525, 454, ...","[1023, 951, 731, 700, 874, 1047, 985, 800, 670...","[3064, 3212, 3661, 4240, 4674, 3223, 3116,

In [ ]:
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return np.array(X), np.array(y)

In [ ]:
# define input sequence
male_sample_data = list(population_m_dict[10101.0].values())
# choose a number of time steps
n_steps = 5
# split into samples
X, y = split_sequence(male_sample_data, n_steps)
# summarize the data
for i in range(len(X)):
	print(X[i], y[i])

[[2603 2565 2517 2472 2178 2392 2478 2312 2296 1999 1735 1502 1513 1170
   765  506  260  159]
 [2605 2567 2499 2371 2183 2294 2483 2331 2267 2099 1754 1524 1489 1214
   807  519  269  163]
 [2585 2557 2480 2294 2145 2183 2455 2345 2247 2198 1767 1559 1470 1251
   853  532  280  166]
 [2557 2551 2466 2244 2098 2087 2425 2348 2244 2241 1827 1584 1447 1284
   898  538  300  169]
 [2508 2561 2446 2204 2018 2047 2359 2379 2244 2277 1879 1622 1429 1310
   927  561  302  172]] [2466 2582 2426 2212 1914 2038 2286 2411 2257 2312 1939 1661 1421 1343
  952  599  309  177]
[[2605 2567 2499 2371 2183 2294 2483 2331 2267 2099 1754 1524 1489 1214
   807  519  269  163]
 [2585 2557 2480 2294 2145 2183 2455 2345 2247 2198 1767 1559 1470 1251
   853  532  280  166]
 [2557 2551 2466 2244 2098 2087 2425 2348 2244 2241 1827 1584 1447 1284
   898  538  300  169]
 [2508 2561 2446 2204 2018 2047 2359 2379 2244 2277 1879 1622 1429 1310
   927  561  302  172]
 [2466 2582 2426 2212 1914 2038 2286 2411 2257 2312

## 3.Train and Predict

In [ ]:
# train data
train_x = X[:6]
train_y = y[:6]
# validation data
#val_x = X[6:10]
#val_y = y[6:10]
# test data
test_x = X[6]
test_y = y[6]

In [ ]:
# univariate lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense

In [ ]:

# reshape from [samples, timesteps] into [samples, timesteps, features]
n_features = 18
train_x = train_x.reshape((train_x.shape[0], train_x.shape[1], n_features))
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps, n_features)))
model.add(Dense(18))
model.compile(optimizer='adam', loss='mse')
# fit model
history = model.fit(train_x, train_y, epochs=200, verbose=0)

# demonstrate prediction
x_input = test_x
x_input = x_input.reshape((1, n_steps, n_features))

prediction_list = []
for iter in range(10):
  prediction = model.predict(x_input, verbose=0)
  prediction_list.append(prediction)

  prediction = prediction.reshape(1,1,18)
  x_input = np.hstack((x_input,prediction)) #add the latest prediction
  x_input = x_input[0][1:].reshape(1,n_steps,n_features)  #delete the first value



In [ ]:
# 10 years prediction on male data
prediction_list

[array([[2272.2795 , 2234.0793 , 2375.492  , 2135.726  , 2007.0968 ,
         1710.2957 , 1792.1785 , 2177.583  , 2138.5156 , 2075.5557 ,
         1865.9525 , 1749.9272 , 1252.2714 , 1403.5824 ,  906.71716,
          616.9245 ,  592.0673 ,  367.1714 ]], dtype=float32),
 array([[2231.4146 , 2219.2227 , 2362.673  , 2136.4385 , 1991.001  ,
         1686.6012 , 1784.619  , 2182.26   , 2134.4934 , 2081.7317 ,
         1842.7004 , 1739.5612 , 1240.917  , 1380.1228 ,  921.55005,
          613.7746 ,  595.8151 ,  374.22577]], dtype=float32),
 array([[2354.1711 , 2177.558  , 2260.203  , 1990.751  , 2012.0596 ,
         1595.3604 , 1883.2499 , 2018.7745 , 1951.3003 , 2029.3512 ,
         1897.0109 , 1664.0958 , 1267.9292 , 1167.8663 , 1026.2167 ,
          659.36554,  402.69867,  454.67682]], dtype=float32),
 array([[2535.6924  , 2641.952   , 2441.7937  , 2032.2294  , 1620.3866  ,
         1880.5629  , 2439.6282  , 2356.0227  , 2179.4954  , 2188.733   ,
         2479.8254  , 1785.1552  , 1896.24

In [ ]:
# prediction for SA3 code(10101) in 2006
prediction_list[4]

array([[2430.6108 , 2594.7026 , 2449.5156 , 2051.239  , 1590.0394 ,
        1961.9562 , 2375.4187 , 2409.0933 , 2281.2615 , 2138.0994 ,
        2418.9304 , 1808.8696 , 1744.927  , 1298.5858 , 1116.6942 ,
         581.9371 ,  191.99825,   96.07537]], dtype=float32)

In [ ]:
# true data
population_m_dict[10101][2006]

[2083,
 2368,
 2524,
 2246,
 1777,
 1702,
 1959,
 2167,
 2452,
 2461,
 2278,
 2407,
 1946,
 1551,
 1188,
 976,
 545,
 317]

In [ ]:
# prediction for SA3 code(10101) in 2011
prediction_list[-1]

array([[2523.6738 , 2160.17   , 2490.6218 , 1973.9579 , 2133.888  ,
        1461.7668 , 2068.44   , 2503.6602 , 2074.698  , 2175.406  ,
        1966.6218 , 1604.0509 ,  816.8614 , 1199.5541 ,  916.45483,
         889.0738 ,  531.8225 ,  588.2581 ]], dtype=float32)

In [ ]:
# true data
population_m_dict[10101][2011]

[2350,
 2338,
 2532,
 2420,
 1806,
 1697,
 1768,
 2204,
 2423,
 2577,
 2555,
 2331,
 2422,
 1950,
 1483,
 1004,
 743,
 490]